# MicroGrid




## MicroGrid System Overview



![](new_design.png)

Advantages of this multi-agent design with autogen:
- Collaborative Problem Solving: The collaboration among the user proxy agent and the assistant agents fosters a cooperative problem-solving environment. The agents can share information and knowledge, allowing them to complement each other's abilities and collectively arrive at better solutions. On the other hand, the Safeguard acts as a virtual adversarial checker, which can perform another safety check pass on the generated code.

- Modularity: The division of tasks into separate agents promotes modularity in the system. Each agent can be developed, tested, and maintained independently, simplifying the overall development process and facilitating code management.

- Memory Management: The UnbiasedCathode agent's role in maintaining memory related to user interactions is crucial. The memory retention allows the agents to have context about a user's prior questions, making the decision-making process more informed and context-aware.



In [1]:
# Install Required Packages
%pip install optiguide
%pip install --force-reinstall -v "openai==0.28.1"


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\nermi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Using pip 23.2.1 from C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip (python 3.11)
  Obtaining dependency information for openai==0.28.1 from https://files.pythonhosted.org/packages/1e/9f/385c25502f437686e4aa715969e5eaf5c2cb5e5ffa7c5cdd52f3c6ae967a/openai-0.28.1-py3-none-any.whl.metadata
  Using cached openai-0.28.1-py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for requests>=2.20 from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Obtaining dependency information for aiohttp 


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\nermi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
import json

In [3]:
with open("api_keys.json", "r") as keyfile:
    api_key = json.load(keyfile)


autogen.oai.ChatCompletion.start_logging()
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            'gpt-3.5-turbo'
        }
    }
)

config_list = [
    {
        'model': 'gpt-3.5-turbo-16k-0613',
        'api_key': api_key
    }]

Now, let's import the source code (loading from URL) and also some training examples (defined as string blow).

In [4]:
# Get the source code of materials example
code_url = "../benchmark/application/microgrid.py"
code = open(code_url, "r").read() # for local files

# DEBUG
# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

from rsome import ro                # import the ro modeling tool
import numpy as np

###DEFINE ALL PARAMETERS
m= 3 #5 timesteps
n= 4 #6 prosumers
Pprod = np.array([[65,45,36,25],
                      [60, 40,30,27],
                       [69,44,31,26]]
                      )
.
.
.

print("############")
print("objective value:",obj_value)
print("Import:",Pimport )
print("Export:",Pexport )
    
print("Charge Battery:",Bcharge )
print("Discharge Battery:",Bdischarge )
print("SoE:",Soe )
print("SoEnext:",soenext )



In [5]:
# In-context learning examples.

example_qa = """
----------
Question: Why is it not recommended to use just one supplier for manifacturer 2?
Answer Code:
```python
z = m.addVars(suppliers, vtype=GRB.BINARY, name="z")
m.addConstr(sum(z[s] for s in suppliers) <= 1, "_")
for s in suppliers:
    m.addConstr(x[s,'manifacturer2'] <= capacity_in_supplier[s] * z[s], "_")
```

----------
Question: What if there's a 13% jump in the demand for light material at manifacturer1?
Answer Code:
```python
light_material_needed_for_manifacturer["manifacturer1"] = light_material_needed_for_manifacturer["manifacturer1"] * (1 + 13/100)
```
"""


'\nexample_qa = """\n----------\nQuestion: Why is it not recommended to use just one supplier for manifacturer 2?\nAnswer Code:\n```python\nz = m.addVars(suppliers, vtype=GRB.BINARY, name="z")\nm.addConstr(sum(z[s] for s in suppliers) <= 1, "_")\nfor s in suppliers:\n    m.addConstr(x[s,\'manifacturer2\'] <= capacity_in_supplier[s] * z[s], "_")\n```\n\n----------\nQuestion: What if there\'s a 13% jump in the demand for light material at manifacturer1?\nAnswer Code:\n```python\nlight_material_needed_for_manifacturer["manifacturer1"] = light_material_needed_for_manifacturer["manifacturer1"] * (1 + 13/100)\n```\n"""\n'

Now, let's create an Unbiased Cathode agent and also a user.

For the Unbiased Cathode agent, we only allow "debug_times" to be 1, which means it can debug its answer once if it encountered errors.

In [ ]:
from SGPChat.unbiasedcathode import UnbiasedCathodeAgent

agent = UnbiasedCathodeAgent(
    name="Unbiased Cathode Material Example",
    source_code=code,
    debug_times=1,
    example_qa=example_qa,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

Now, let's create a user's question.

In [7]:
user.initiate_chat(agent, message="Are we able to cover our demand?")

user (to OptiGuide Material Example):

Are we able to cover our demand?

--------------------------------------------------------------------------------
OptiGuide Material Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Material Example):

```python
# Check if demand is covered
if model.status == GRB.OPTIMAL:
    print("Demand is covered. Optimal cost:", model.objVal)
else:
    print("Demand is not covered.")
```

--------------------------------------------------------------------------------
OptiGuide Material Example (to safeguard):


--- Code ---
# Check if demand is covered
if model.status == GRB.OPTIMAL:
    print("Demand is covered. Optimal cost:", model.objVal)
else:
    print("Demand is not covered.")

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to OptiGuide Material Example):

SAFE

-------------

In [8]:
user.initiate_chat(agent, message="What is the material usage if the quality threshold increases by 10%?")

user (to OptiGuide Material Example):

What is the material usage if the quality threshold increases by 10%? To remove a constraint, use the call model.remove(model.getConstrs()[index])

--------------------------------------------------------------------------------
OptiGuide Material Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Material Example):

```python
# Increase quality threshold by 10%
QUALITY_THRESHOLD *= 1.1

# Update quality constraint
model.getConstrByName('quality constraint').rhs = QUALITY_THRESHOLD

# Re-optimize model
model.optimize()

# Print new material usage
if model.status == GRB.OPTIMAL:
    for i in material_quality.keys():
        print(f'new material {i} usage: {x[i].x}')
else:
    print("Not solved to optimality. Optimization status:", model.status)
```
Explanation:
To increase the quality threshold by 10%, we multiply the current value (`QUALITY_THRESHOLD`) by 1.1.

In [ ]:
'''
Problems: Agent Writer does not seem to be able to interpret the results of the optimization correctly. 
Approach: Try in-context learning there as well 

'''